In [ ]:
import sqlite3



In [ ]:
db_name = "db/serenity_soother.db"

# get a connection to the database
conn = sqlite3.connect(db_name)

# get a cursor object
cursor = conn.cursor()

# retrieve all the tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# print the tables
print("Tables in the database:")
for table in tables:
    print(table[0])

# get the user table
scripts = cursor.execute("SELECT * FROM scripts;")
scenes = cursor.execute("SELECT * FROM scenes;")
scenes = scenes.fetchall()
scripts = scripts.fetchall()


In [ ]:
import pandas as pd

# create a pandas dataframe from the users
scenes_df = pd.DataFrame(scripts, columns=[i[0] for i in cursor.description])


In [ ]:
len(scenes_df)

In [ ]:
scripts_df = pd.DataFrame(scripts, columns=[i[0] for i in cursor.description])

In [ ]:
import pandas as pd


In [1]:

from app.services.nft_service import DataUploader


base_path = "/Users/mohameddiomande/Desktop/1792x1024/data"
data_uploader = DataUploader(directory=base_path)
parsed_data = data_uploader.upload_all_media_in_parallel(
    "/Users/mohameddiomande/Desktop/1792x1024/data"
)


# print(parsed_data.head())
from datasets import Dataset, load_dataset, DatasetDict

class DataUploader:
    # Existing methods...

    def convert_to_hf_dataset(self, df):
        """
        Converts a pandas DataFrame to a Hugging Face dataset.

        Args:
            df (pd.DataFrame): The DataFrame to convert.

        Returns:
            Dataset: A Hugging Face dataset created from the DataFrame.
        """
        # Convert DataFrame to a Hugging Face Dataset
        hf_dataset = Dataset.from_pandas(df)
        return hf_dataset

    def upload_to_huggingface(self, dataset, dataset_name, save_path):
        """
        Uploads a dataset to Hugging Face Datasets.

        Args:
            dataset (Dataset): The Hugging Face dataset to upload.
            dataset_name (str): The name of the dataset to create or update.
            save_path (str): The path to save the dataset locally before pushing to Hugging Face.

        Returns:
            str: A URL or identifier for the uploaded dataset.
        """
        # You must authenticate to Hugging Face Hub (assuming you have HF Hub CLI installed)
        # os.system("huggingface-cli login")
        
        # Save the dataset locally
        dataset.save_to_disk(save_path)
        
        # Push to Hugging Face Datasets (adjust this command according to your authentication and access rights)
        # os.system(f"cd {save_path} && huggingface-cli repo create {dataset_name} --type dataset && git push")
        
        # The above step assumes you have your Hugging Face credentials set up
        return f"Dataset uploaded to Hugging Face under the name {dataset_name}"

    def upload_data(self, base_path, dataset_name):
        """
        Processes and uploads data from a given directory to Hugging Face Datasets.

        Args:
            base_path (str): The base path where the files are located.
            dataset_name (str): The name for the dataset to be created on Hugging Face.

        Returns:
            str: A confirmation message with the dataset URL or name.
        """
        exploded_df = self.explode_list_columns(base_path)
        hf_dataset = self.convert_to_hf_dataset(exploded_df)
        save_path = f"./hf_datasets/{dataset_name}"  # Local save path for the dataset
        os.makedirs(save_path, exist_ok=True)
        return self.upload_to_huggingface(hf_dataset, dataset_name, save_path)



2024-05-19 11:10:16,195 - DataUploader - ERROR - Failed to load /Users/mohameddiomande/Desktop/1792x1024/data/61/tickets.csv


Processed /Users/mohameddiomande/Desktop/1792x1024/data/61/audio/7cae9bfd-6687-4922-be64-a90a97506038/segment_1.mp3
Processed /Users/mohameddiomande/Desktop/1792x1024/data/61/audio/7cae9bfd-6687-4922-be64-a90a97506038/segment_2.mp3
Processed /Users/mohameddiomande/Desktop/1792x1024/data/61/audio/7cae9bfd-6687-4922-be64-a90a97506038/segment_3.mp3
Processed /Users/mohameddiomande/Desktop/1792x1024/data/61/audio/7cae9bfd-6687-4922-be64-a90a97506038/segment_4.mp3
Processed /Users/mohameddiomande/Desktop/1792x1024/data/61/audio/7cae9bfd-6687-4922-be64-a90a97506038/segment_5.mp3
Processed /Users/mohameddiomande/Desktop/1792x1024/data/61/audio/7cae9bfd-6687-4922-be64-a90a97506038/segment_6.mp3
Processed /Users/mohameddiomande/Desktop/1792x1024/data/61/audio/7cae9bfd-6687-4922-be64-a90a97506038/segment_7.mp3
Processed /Users/mohameddiomande/Desktop/1792x1024/data/61/audio/7cae9bfd-6687-4922-be64-a90a97506038/segment_8.mp3
Processed /Users/mohameddiomande/Desktop/1792x1024/data/61/audio/7cae9bf

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
len(parsed_data)

In [ ]:
import pandas as pd

# Assuming parsed_data_df is your DataFrame and the list columns are 'prompt', 'caption', 'image', 'audio'
# Create a new DataFrame by applying a lambda function to each row, which converts it into a DataFrame
# and then concatenates these DataFrames:
parsed_data_df = pd.DataFrame(parsed_data)

import pandas as pd

# Function to pad lists to the same length
def pad_lists_to_max_length(row):
    max_length = max(len(row['prompt']), len(row['caption']), len(row['image']), len(row['audio']))
    return {
        'prompt': row['prompt'] + [None]*(max_length - len(row['prompt'])),
        'caption': row['caption'] + [None]*(max_length - len(row['caption'])),
        'image': row['image'] + [None]*(max_length - len(row['image'])),
        'audio': row['audio'] + [None]*(max_length - len(row['audio']))
    }

# Apply the padding function and then explode the DataFrame
exploded_df = pd.concat(
    parsed_data_df.apply(lambda x: pd.DataFrame(pad_lists_to_max_length(x)), axis=1).tolist(), ignore_index=True
)

# Now check the resulting DataFrame
print(exploded_df.head())


In [ ]:
exploded_df.to_csv("data.csv", index=False)

In [ ]:

parsed_data_df = pd.DataFrame(parsed_data)
exploded_df = parsed_data_df.explode('prompt')

# Assuming alignment, the following columns will align naturally:
exploded_df['caption'] = parsed_data_df['caption'].explode()
exploded_df['image'] = parsed_data_df['image'].explode()
exploded_df['audio'] = parsed_data_df['audio'].explode()

# Now check the resulting DataFrame
print(exploded_df.head())

In [ ]:
import time

In [ ]:
import pandas as pd
from app.routers.chat import chat_service, generate_prompt
from app.services.prompt_generator_service import PromptGenerator
import asyncio

In [ ]:
gen = PromptGenerator(chat=chat_service)

In [ ]:
parsed_data_df  =pd.read_csv("data_best.csv")

# combine the prompt and caption columns
parsed_data_df['prompt_caption'] = parsed_data_df['prompt'] + "\n\n" + parsed_data_df['caption']


example_pairs = list(zip(parsed_data_df['prompt'], parsed_data_df['caption']))[0:20]


In [ ]:
results = gen.generate_parallel(
    num_prompts=20,
    example_pairs=example_pairs,
    max_workers=4,
    generate_prompt=generate_prompt,
    batch_size=3
)

for result in results:
    print(result)

In [ ]:
results

In [ ]:

generate = gen.generate_parallel(
    num_prompts=20,
    example_pairs=example_pairs,
    max_workers=1,
    generate_prompt=generate_prompt,
    batch_size=1    
)

In [ ]:
import json
from typing import Optional
import pandas as pd

def generate_batch_examples(
    data: pd.DataFrame,
    filename: str = "batch_requests",
    model: str = "gpt-3.5-turbo-0125",
    max_tokens: int = 1000,
    api_endpoint: str = "/v1/chat/completions"
) -> None:
    """
    Generate a batch file for API requests from a DataFrame.
    
    Parameters:
        data (pd.DataFrame): DataFrame containing the columns for system, user messages.
        filename (str): Name of the file to save the batch requests.
        model (str): Model identifier for the API request.
        max_tokens (int): Maximum number of tokens for the response.
        api_endpoint (str): API endpoint for the request.
    """
    with open(f"{filename}.jsonl", "w") as file:
        for index, row in data.iterrows():
            request_data = {
                "custom_id": f"request-{index}",
                "method": "POST",
                "url": api_endpoint,
                "body": {
                    "model": model,
                    "messages": [],
                    "max_tokens": max_tokens
                }
            }
            
            # Append system message if available
            if 'system_message' in row:
                request_data['body']['messages'].append({"role": "system", "content": row['system_message']})
            
            # Append user and assistant messages
            if 'user_message' in row and 'assistant_message' in row:
                request_data['body']['messages'].append({"role": "user", "content": row['user_message']})
                request_data['body']['messages'].append({"role": "assistant", "content": row['assistant_message']})
            
            file.write(json.dumps(request_data) + "\n")

    print(f"Batch requests saved to {filename}.jsonl")

# Example of calling the function
data = pd.DataFrame({
    'system_message': ["You are a helpful assistant.", "You are an unhelpful assistant."],
    'user_message': ["Hello world!", "Hello world!"],
    'assistant_message': ["How can I assist you today?", "I'm not sure how to help."]
})
generate_batch_examples(data)

In [ ]:
import pandas as pd
import time

# Assuming parsed_data_df is your DataFrame with columns 'prompt' and 'caption'
def process_in_batches(df, batch_size):
    for i in range(0, len(df), batch_size):
        # Extract the current batch as a DataFrame slice
        batch_df = df[i:i+batch_size]
        
        # Create 'prompt_caption' for the batch
        batch_df['prompt_caption'] = batch_df['prompt'] + "\n\n" + batch_df['caption']
        
        # Process each prompt in the current batch and create a list of dictionaries
        results = [{
            "index": index,  # Add an index to keep track of the original row number
            'prompt': row['prompt'], 
            'caption': row['caption'],
            'prompt_caption': row['prompt_caption'],
            'category': chat_service.generate_category(row['prompt_caption'])
        } for index, row in batch_df.iterrows()]
        
        # Convert the results to a DataFrame
        results_df = pd.DataFrame(results)
        
        # Optionally: save DataFrame to a file, database, etc.
        results_df.to_csv(f'batch_{i//batch_size}.csv', index=False)  # Example to save to CSV
        
        print(f"Processed batch {i//batch_size + 1}")
        print(results_df)
        
        # Sleep for 10 seconds before processing the next batch
        time.sleep(10)

# Call the function with the entire DataFrame and a batch size of 10
process_in_batches(parsed_data_df, 10)


In [ ]:

results = [{'prompt_caption': prompt, 'category': chat_service.generate_category(prompt)} for prompt in prompt_caption]

results_df = pd.DataFrame(results)

In [ ]:
results_df

In [ ]:
cat

In [ ]:
parsed_data  = parsed_data_df.to_dict(orient="records")

In [ ]:
# from typing import List, Optional, Tuple
# from concurrent.futures import ThreadPoolExecutor
# from app.services.cloud_service import CloudManager
# from app.callbacks.streaming import StreamingHandler
# from app.response.system import ReplyChainSystem
# from app.services.chat_service import AI
# from app.chain_tree.schemas import Chain
# import uuid

# class PromptService:
#     def __init__(self,
#                  model_name: Optional[str],
#                  callback: Optional[StreamingHandler] = None,
#                  credentials: Optional[dict] = None,
#                  segment_delimiter: Optional[str] = ".",
#                  name: Optional[str] = None,
#                  technique: Optional[object] = None,
#                  upload: Optional[bool] = None,
#                  path: Optional[str] = None,
#                  with_responses: Optional[bool] = False,
#                  storage: Optional[str] = None,
#                  max_tokens: Optional[int] = 4096,
#                  target_tokens: Optional[int] = 16385,
#                  verbose: Optional[bool] = False,
#                  create: Optional[bool] = False,
#                  play: Optional[bool] = False,
#                  show: Optional[bool] = False,
#                  process_media: Optional[bool] = False,
#                  internal: Optional[bool] = False,
#                  audio_func: Optional[bool] = False,
#                  stop: Optional[str] = None,
#                  provider: Optional[str] = "openai",
#                  convert: Optional[bool] = False,
#                  subdirectory: Optional[str] = None):
        
#         self.stop = stop
#         self.name = name
#         self.path = path
#         self.build = create
#         self.upload = upload
#         self.verbose = verbose
#         self.convert = convert
#         self.storage = storage
#         self.provider = provider
#         self.subdirectory = subdirectory
#         self.with_responses = with_responses

#         if subdirectory is None:
#             subdirectory = str(uuid.uuid4())

#         self.subdirectory = subdirectory

#         self.prompt_manager = CloudManager(credentials=credentials, directory=storage)

#         self.reply_chain_system = ReplyChainSystem(
#             name=name,
#             register_synthesis_technique=technique,
#             verbose=verbose,
#         )
#         self.generator = Generator()
#         self.callback = (
#             callback
#             if callback
#             else StreamingHandler(
#                 segment_delimiter=segment_delimiter,
#             )
#         )
#         self.chat = AI(
#             storage=storage,
#             callbacks=[self.callback],
#             model_name=model_name,
#             prompt_manager=self.prompt_manager,
#             max_tokens=max_tokens,
#             create=create,
#             play=play,
#             process_media=process_media,
#             internal=internal,
#             show=show,
#             audio_func=audio_func,
#             provider=provider,
#             subdirectory=subdirectory,
#             target_tokens=target_tokens,
#         )

#     async def generate_prompt_parts(
#         self,
#         prompt: Optional[str] = None,
#         response: Optional[str] = None,
#         conversation_id: Optional[str] = None,
#         use_basic_truncation: Optional[bool] = False,
#         **kwargs,
#     ) -> Chain:
#         conversation_history = await self.reply_chain_system.prepare_conversation_history(
#             prompt, response, **kwargs
#         )

#         truncated_history = self.chat._process_conversation_history(
#             conversation_history,
#             prompt=prompt,
#             use_basic_truncation=use_basic_truncation,
#             verbose=self.verbose,
#         )

#         return await self.chat(truncated_history, conversation_id, self.stop)

#     async def _create_prompt_and_embedding(
#         self,
#         text: str,
#         conversation_id: str,
#         prompt: Optional[str] = None,
#         upload: Optional[bool] = None,
#         **kwargs,
#     ) -> None:
#         embedding = await self.chat.generate_embeddings(text)
#         await self.prompt_manager.create_prompt(
#             prompt=prompt,
#             prompt_parts=text.split("\n\n"),
#             id=conversation_id,
#             embedding=embedding,
#             upload=upload,
#             **kwargs,
#         )

#     async def generate_prompt(
#         self,
#         prompt: Optional[str] = None,
#         response: Optional[str] = None,
#         **kwargs,
#     ) -> str:
#         text = await self.generator._generic_creation(
#             prompt=prompt,
#             response=response,
#             generation_function=self.generate_prompt_parts,
#             creation_function=self._create_prompt_and_embedding,
#             upload=self.upload,
#             **kwargs,
#         )
#         return text

#     async def run_thread(self, prompt, **kwargs) -> str:
#         return await self.generator._generic_prompt_creation(
#             prompt=prompt,
#             generation_function=self.generate_prompt,
#             creation_function=self._create_prompt_and_embedding,
#             **kwargs,
#         )

#     async def run_chat(self, **kwargs) -> str:
#         return await self.generator.run_chat(
#             generate_prompt=self.generate_prompt_parts,
#             chat=self.chat,
#             **kwargs,
#         )

#     async def _create_future_tasks(
#         self,
#         executor: ThreadPoolExecutor,
#         prompt: str,
#         answer: str = None,
#         parent_id: str = None,
#         answer_split: bool = False,
#         mode: str = "run",
#         **kwargs, 
#     ):
#         futures = []
#         if answer_split:
#             responses = answer.split("\n\n")
#         else:
#             responses = [answer]

#         for index, response in enumerate(responses):
#             task_parent_id = f"{parent_id}_{index}" if answer_split else parent_id

#             if mode == "run":
#                 future = executor.submit(self.run_thread, prompt=prompt, **kwargs)

#             elif mode == "run_chat":
#                 future = executor.submit(self.run_chat, **kwargs)

#             else:
#                 future = executor.submit(
#                     self.generate_prompt,
#                     prompt,
#                     response,
#                     **kwargs,
#                 )
#             futures.append(future)
#         return futures

#     async def _get_unique_prompt(self, example_pairs: List, generated_prompts: set):
#         parent_id = str(uuid.uuid4())
#         prompt, answer = example_pairs.pop()
#         while (prompt, answer, parent_id) in generated_prompts:
#             prompt, answer, parent_id = example_pairs.pop()
#         generated_prompts.add((prompt, answer, parent_id))
#         return prompt, answer, parent_id

#     async def run_interactive(
#         self,
#         mode: str = "run",
#         answer_split: bool = False,
#         max_workers: int = 1,
#         batch_size: int = 4,
#         example_pairs: Optional[List[Tuple[str, str]]] = None,
#         with_responses: Optional[bool] = True,
#     ) -> None:
#         num_prompts = len(example_pairs)
#         generated_prompts = set()
#         total_batches = (num_prompts + batch_size - 1) // batch_size
#         valid_count = 0

#         for batch_num in range(total_batches):
#             print(f"Generating batch {batch_num + 1} of {total_batches}")

#             with ThreadPoolExecutor(max_workers=max_workers) as executor:
#                 futures = []

#                 for _ in range(min(batch_size, num_prompts - valid_count)):
#                     prompt, answer, parent_id = await self._get_unique_prompt(
#                         example_pairs, generated_prompts
#                     )
#                     if with_responses:
#                         futures.extend(
#                             self._create_future_tasks(
#                                 executor,
#                                 prompt,
#                                 answer,
#                                 parent_id,
#                                 answer_split,
#                                 mode,
#                             )
#                         )
#                     else:
#                         futures.extend(
#                             self._create_future_tasks(
#                                 executor,
#                                 prompt,
#                                 None,
#                                 parent_id,
#                                 answer_split,
#                                 mode,
#                             )
#                         )

#                 for future in futures:
#                     await future.result()

#             valid_count += len(futures)

#         print(f"Generated {valid_count} prompts.")
